In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    #set the derivatives
    
    #our equations is d^2y/dx^2
    
    #so we can write 
    #dydx =z
    #dzdx=-y
    
    # We will set y = y[0]
    # We will set z = y[1]
    
    #declare an array
    y_derivs=np.zeros(2)
    
    #set dy/dx =z
    y_derivs[0] = y[1]
    
    #set dfdx = -y
    y_derivs[1] = -1.*y[0]
    
    return y_derivs

## Define the 4th order RK method

In [ ]:
#nv number of varbiable
#xi x and time i
#yi the values of y and z 
#h is a step
def rk4(dydx,xi,yi,nv,h):
    #declare k? array
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi +0.5*h
     
        
    #define x at 1 step    
    x_ipo = xi +h

    y_temp = np.zero(nv)
    
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    y_tem[:] = yi[:] +0.5*k1[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k2[:] = h*y_derivs[:]
    
    
    #get k3 values
    y_temp[:] =yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipo,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo, y_temp)
    k4[:]=h*y_derivs
    
    #advance y by a step h
    yipo = yi=(k1 + 2*k2 + 2*k3 + k4)/6.0
    
    return yipo

## Define an adaptive step size driver for RK4

In [ ]:
def kr4_mv_ad(dydx,xi,yi,h,tol):
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a max # of iterations
    imax = 10000
    
    #set a iteration variable
    i=0
    
    #create an error   
    Delta = np.full(nv,2*tol)
    
    h_step = h
    
    #adjust the step
    
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by taking one step of size h
        y_2 = rk4_mv_core(dydx, x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol>1.0):  # this means it to big
            h_step *=SAFETY * (Delta.max()/tol)**(-0.25)
            
            
        #check iterations
        if(i>imax):
            print("Too many iterations inrk4_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        i +=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Dealta.max()/tol)**(-0.9), h_step * H_NEW_FACT)
    
    #return the answer
    return y_2,h_new,h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dfdx,a,b,y_a,tol):
    
    xi = a
    yi = y_a.copy()
    
    #An initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 1000
    
    #set an iterations variable
    i =0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)  # y_a has the size of our boundery conditions